# CHAPTER 39 - DECORATORS

## THE BASICS 

In [4]:
def decorator(cls):
    class Wraper:
        def __init__(self, *args):
            self.wrapper = cls(*args)
        def __getattr__(self, name):
            return getattr(self.wrapper, name)
    return Wraper

In [5]:
@decorator
class C:
    def __init__(self, x, y):
        self.attr = 'spam'

In [6]:
x = C(6,7)

In [8]:
x.attr

'spam'

## CODING FUNCTIONS DECORATORS

In [10]:
class tracer:
    def __init__(self, func):
        self.calls = 0
        self.func = func
    def __call__(self, *args):
        self.calls += 1
        print(f'calls {self.calls} to {self.func.__name__}')

In [11]:
@tracer
def spam(a, b, c):
    print(a + b + C)

In [12]:
spam(1, 2, 3)

calls 1 to spam


In [13]:
spam('a', 'b', 'c')

calls 2 to spam


In [14]:
spam.calls

2

## DECORATOR STATE RETENTION OPTIONS

In [26]:
class tracer:
    def __init__(self, func):
        self.calls = 0
        self.func = func
    def __call__(self, *args, **kwargs):
        self.calls += 1
        print(f'call {self.calls} to {self.func.__name__}')
        return self.func(*args, **kwargs)

In [2]:
@tracer
def spam(a, b, c):
    print(a + b + c)

In [4]:
@tracer
def eggs(x, y):
    print(x ** y)

In [5]:
spam(1, 2, 3)

call 1 to spam
6


In [6]:
spam(a=4, b=5, c=6)

call 2 to spam
15


In [7]:
eggs(2, 16)

call 1 to eggs
65536


In [8]:
eggs(4, y=4)

call 2 to eggs
256


In [10]:
def tracer(func):
    calls = 0
    def wrapper(*args, **kwargs):
        nonlocal calls
        calls += 1
        print(f'call {calls} to {func.__name__}')
        return func(*args, **kwargs)
    return wrapper

In [11]:
@tracer
def spam(a, b, c):
    print(a + b + c)

In [12]:
@tracer
def eggs(x, y):
    print(x ** y)

In [13]:
spam(1, 2, 3)

call 1 to spam
6


In [14]:
spam(a=4, b=5, c=6)

call 2 to spam
15


In [15]:
eggs(2, 16)

call 1 to eggs
65536


In [16]:
eggs(4, y=4)

call 2 to eggs
256


## CLASS BLUNDERS I: DECORATING METODS 

In [17]:
class Person:
    def __init__(self, name, pay):
        self.name = name
        self.pay = pay

    @tracer
    def giveRaise(self, percent):
        self.pay *= (1. + percent)

    @tracer
    def lastName(self):
        return self.name.split()[-1]

In [27]:
bob = Person('Bob Smith', 50000)

In [28]:
bob.giveRaise(.25)

call 3 to giveRaise


In [29]:
print(bob.lastName())

call 3 to lastName
Smith


In [30]:
bob.pay

62500.0

In [31]:
bob.giveRaise(.25)

call 4 to giveRaise


In [32]:
print(bob.lastName())

call 4 to lastName
Smith


## TIMING CALLS 

In [6]:
import time, sys

In [53]:
class timer:
    def __init__(self, func):
        self.func = func
        self.alltime = 0
    def __call__(self, *args, **kargs):
        start = time.time()
        result = self.func(*args, **kargs)
        elapsed = time.time() - start
        self.alltime += elapsed
        print(f'{self.func.__name__}: {self.alltime}')
        return result

In [54]:
@timer
def listcomp(N):
    return [x * 2 for x in range(N)]

In [71]:
@timer
def mapcall(N):
    return list(map((lambda x: x * 2), range(N)))

In [72]:
result = listcomp(5) # Time for this call, all calls, return value
listcomp(50000)
listcomp(500000)
listcomp(1000000)
print(result)
print('allTime = %s' % listcomp.alltime) # Total time for all listcomp calls

listcomp: 0.3282938003540039
listcomp: 0.331326961517334
listcomp: 0.3762953281402588
listcomp: 0.45629239082336426
[0, 2, 4, 6, 8]
allTime = 0.45629239082336426


In [73]:
result = mapcall(5)
mapcall(50000)
mapcall(500000)
mapcall(1000000)
print(result)
print('allTime = %s' % mapcall.alltime)

mapcall: 0.0
mapcall: 0.006997346878051758
mapcall: 0.0820014476776123
mapcall: 0.2289738655090332
[0, 2, 4, 6, 8]
allTime = 0.2289738655090332


## ADDING DECORATOR ARGUMENTS

In [1]:
def timer(label='', trace=True):
    class Timer:
        def __init__(self, func):
            self.func = func
            self.alltime = 0
        def __call__(self, *args, **kargs):
            import time
            start = time.time()
            result = self.func(*args, *kargs)
            elapsed = time.time() - start
            self.alltime += elapsed
            if trace:
                format = '%s %s: %.5f, %.5f'
                values = (label, self.func.__name__, elapsed, self.alltime)
                print(format % values)
            return result
    return Timer

In [2]:
@timer(label='[CCC]==>')
def listcomp(N):
    return [x * 2 for x in range(N)]

In [3]:
@timer(trace=True, label='[MMM]==>')
def mapcall(N):
    return list(map((lambda x: x * 2), range(N)))

In [4]:
for func in (listcomp, mapcall):
    result = func(5)
    func(50000)
    func(500000)
    func(1000000)
    print(result)
    print('allTime = %s\n' % func.alltime) 
print('**map/comp = %s' % round(mapcall.alltime / listcomp.alltime, 3))

[CCC]==> listcomp: 0.00000, 0.00000
[CCC]==> listcomp: 0.00300, 0.00300
[CCC]==> listcomp: 0.05501, 0.05801
[CCC]==> listcomp: 0.09552, 0.15353
[0, 2, 4, 6, 8]
allTime = 0.15353131294250488

[MMM]==> mapcall: 0.00000, 0.00000
[MMM]==> mapcall: 0.00700, 0.00700
[MMM]==> mapcall: 0.06702, 0.07402
[MMM]==> mapcall: 0.13695, 0.21097
[0, 2, 4, 6, 8]
allTime = 0.21096515655517578

**map/comp = 1.374


## CODING CLASS DECORATORS

In [6]:
instances = {}
def singleton(aClass):
    def onCall(*args, **kwargs):
        if aClass not in instances:
            instances[aClass] = aClass(*args, **kwargs)
        return instances[aClass]
    return onCall

In [7]:
@singleton
class Person:
    def __init__(self, name, hours, rate):
        self.name = name
        self.hours = hours
        self.rate = rate
    def pay(self):
        return self.hours * self.rate

In [8]:
@singleton
class Spam:
    def __init__(self, val):
        self.attr = val    

In [9]:
bob = Person('Bob', 40, 10)
print(bob.name, bob.pay())

Bob 400


In [10]:
sue = Person('Sue', 50, 20)
print(sue.name, sue.pay())

Bob 400


In [11]:
X = Spam(val=42)

In [12]:
Y = Spam(99)

In [13]:
print(X.attr, Y.attr)

42 42


In [14]:
def singleton(aClass):
    instance = None
    def onCall(*args, **kwargs):
        nonlocal instance
        if instance == None:
            instance = aClass(*args, **kwargs)
        return instance
    return onCall

In [15]:
def singleton(aClass):
    def onCall(*args, **kwargs):
        if onCall.instance == None:
            onCall.instance = aClass(*args, **kwargs)
        return onCall.instance
    onCall.instance = None
    return onCall

In [16]:
class singleton:
    def __init__(self, aClass):
        self.aClass = aClass
        self.instance = None
    def __call__(self, *args, **kwargs):
        if self.instance == None:
            self.instance = self.aClass(*args, **kwargs)
        return self.instance

## TRACING OBJECTS INTERFACES